## Anlyze paired images in the folder 

The function processes pairs of `.tif`  images from a specified folder using a provided analysis function. Each pair inside the folder consists of two consecutive images.
The second image in each pair contains intensity fluorescence data, and the first image contains fitted data including mean lifetimes and chi-square values. 
Important is to ensure that images are named accordingly and are arranged consecutively.
The `in_folder_loop` function generates results and reports for each pair of images and can optionally save the generated images.


### Operational Modes

The function has two modes of operation:
1. **Tabular Data Only**: Generates only tabular data, which is suitable for analysis.
2. **Tabular Data and Images**: Generates both tabular data and images, providing a visual representation of the results.

## Add package to python path and import function (s)

In [1]:
import sys
import os

# Manually set the parent directory path
notebook_dir = os.getcwd()  # This gets the current working directory of the notebook
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the parent directory to the system path
sys.path.insert(0, parent_dir)

#import an example function from package 
from flim_processing import analyze_single_fitted_channel,in_folder_loop

## Provide the path to the folder with paired images 

In [2]:
from pathlib import Path
folder_path=Path(r"G:\SP8_FLIM\Treatments\Aldometanib\2024-02-02\wt_Aldo_0")

## Analyze files inside the folder with tabular output

In [3]:
# Example of calling the function with saving images
result_per_folder,report_per_folder=in_folder_loop(folder_path,selected_analysis_function=analyze_single_fitted_channel)
report_per_folder

,Mean_fluor_intensity,Mean_tau,Mean_STD,Number_of_objects,Initial_number_of_objects,%_of_object_remained,File Name
0,398.949045,848.605096,NaN,1,63,1.587302,NT-1
0,531.441966,1009.843915,33.133893,23,35,65.714286,NT-2
0,310.420765,927.409836,NaN,1,107,0.934579,NT-3
0,377.193525,908.282458,41.976616,12,45,26.666667,NT-4
0,331.019869,940.933975,29.139302,5,63,7.936508,NT-5


## Analyze files inside the folder with both tabular output and images
stored in locally created 'Output' folder 

In [7]:
# Create an output folder to store all the images 
output_folder_path = Path('Outputs')
output_folder_path.mkdir(exist_ok=True)
selected_analysis_function=analyze_single_fitted_channel

# Set the date variable(which will be the name of images, used for identification)
date = "Random_date_that_is_a_variable"

# Example of calling the function with saving images
result, report = in_folder_loop(folder_path, selected_analysis_function,
                                save_images=True, output_folder_path=output_folder_path, date=date)

report

C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Example for functions\helper_functions.py:83: UserWarning: C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Example for functions\Outputs\param_map_NT-1.tif is a low contrast image
  imsave(output_folder_path / image_name, multichannel_image[..., -1])
C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Example for functions\helper_functions.py:83: UserWarning: C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Example for functions\Outputs\param_map_NT-2.tif is a low contrast image
  imsave(output_folder_path / image_name, multichannel_image[..., -1])
C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Example for functions\helper_functions.py:83: UserWarning: C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Example for functions\Outputs\param_map_NT-3.tif is a low contrast image
  imsave(output_folder_path / image_name, multichannel_image[..., -1])
C:\Users\med\FLIMrepo\granules-flim-analysis\notebooks\Exampl

,Mean_fluor_intensity,Mean_tau,Mean_STD,Number_of_objects,Initial_number_of_objects,%_of_object_remained,File Name
0,398.949045,848.605096,NaN,1,63,1.587302,NT-1
0,531.441966,1009.843915,33.133893,23,35,65.714286,NT-2
0,310.420765,927.409836,NaN,1,107,0.934579,NT-3
0,377.193525,908.282458,41.976616,12,45,26.666667,NT-4
0,331.019869,940.933975,29.139302,5,63,7.936508,NT-5


In [ ]:
#example of saving the files with reports locally 
result_per_folder.to_csv("result_per_folder.csv")
report_per_folder.to_csv("report_per_folder.csv")